In [1]:
import skfuzzy as fuzz
import skfuzzy.control as ctrl
import numpy as np

In [2]:
# 6 inputs
wt = ctrl.Antecedent(np.arange(0, 181, 1), 'waiting_time')
wt['Short'] = fuzz.trapmf(wt.universe, [0, 0, 15, 35])
wt['Medium'] = fuzz.trimf(wt.universe, [15, 35, 55])
wt['Long'] = fuzz.trapmf(wt.universe, [35, 55, 180, 999])

rt = ctrl.Antecedent(np.arange(0, 181, 1), 'riding_time')
rt['Short'] = fuzz.trapmf(rt.universe, [0, 0, 15, 35])
rt['Medium'] = fuzz.trimf(rt.universe, [15, 35, 55])
rt['Long'] = fuzz.trapmf(rt.universe, [35, 55, 180, 999])

load = ctrl.Antecedent(np.arange(0, 16, 1), 'loading')
load['Low'] = fuzz.trapmf(load.universe, [0, 0, 3, 7])
load['Medium'] = fuzz.trimf(load.universe, [3, 7, 11])
load['High'] = fuzz.trapmf(load.universe, [7, 11, 15, 15])

td = ctrl.Antecedent(np.arange(0, 16, 1), 'travelling_distance')
td['Close'] = fuzz.trapmf(td.universe, [0, 0, 4, 8])
td['Middle'] = fuzz.trimf(td.universe, [4, 8, 12])
td['Far'] = fuzz.trapmf(td.universe, [8, 12, 15, 15])

hcaw = ctrl.Antecedent(np.arange(1, 16, 1), 'hall_call_area_weight')
hcaw['LF'] = fuzz.trapmf(hcaw.universe, [1, 1, 2, 4])
hcaw['LM'] = fuzz.trimf(hcaw.universe, [2, 4, 7])
hcaw['Close'] = fuzz.trimf(hcaw.universe, [4, 7, 11])
hcaw['HM'] = fuzz.trimf(hcaw.universe, [7, 11, 13])
hcaw['HF'] = fuzz.trapmf(hcaw.universe, [11, 13, 15, 15])

daw = ctrl.Antecedent(np.arange(1, 16, 1), 'destination_area_weight')
daw['LF'] = fuzz.trapmf(daw.universe, [1, 1, 2, 4])
daw['LM'] = fuzz.trimf(daw.universe, [2, 4, 7])
daw['Close'] = fuzz.trimf(daw.universe, [4, 7, 11])
daw['HM'] = fuzz.trimf(daw.universe, [7, 11, 13])
daw['HF'] = fuzz.trapmf(daw.universe, [11, 13, 15, 15])

# 1 output
priority = ctrl.Consequent(np.linspace(0, 1, 500), 'priority')
priority['Low'] = fuzz.trimf(priority.universe, [0, 0, 0.5])
priority['Medium'] = fuzz.trimf(priority.universe, [0, 0.5, 1])
priority['High'] = fuzz.trimf(priority.universe, [0.5, 1, 1])

In [3]:
# rules
rules = []
r1 = ctrl.Rule(wt['Short'], priority['High']%0.7)
r2 = ctrl.Rule(wt['Medium'], priority['Medium']%0.7)
r3 = ctrl.Rule(wt['Long'], priority['Low'])
rules.append([r1, r2, r3])

r4 = ctrl.Rule(rt['Short'], priority['High']%0.4)
r5 = ctrl.Rule(rt['Medium'], priority['Medium']%0.4)
r6 = ctrl.Rule(rt['Long'], priority['Low']%0.4)
rules.append([r4, r5, r6])

r7 = ctrl.Rule(load['Low'], priority['High']%0.5)
r8 = ctrl.Rule(load['Medium'], priority['Medium'])
r9 = ctrl.Rule(load['High'], priority['Low']%0.6)
rules.append([r7, r8, r9])

r10 = ctrl.Rule(td['Close'], priority['High']%0.05)
r11 = ctrl.Rule(td['Middle'], priority['Medium']%0.05)
r12 = ctrl.Rule(td['Far'], priority['Low']%0.05)
rules.append([r10, r11, r12])

r13 = ctrl.Rule(hcaw['Close'], priority['Medium']%0.7)
r14 = ctrl.Rule(hcaw['LM'], priority['High'])
r15 = ctrl.Rule(hcaw['HM'], priority['Medium']%0.5)
r16 = ctrl.Rule(hcaw['LF'], priority['High'])
r17 = ctrl.Rule(hcaw['HF'], priority['Low']%0.5)
rules.append([r13, r14, r15, r16, r17])

r18 = ctrl.Rule(daw['Close'], priority['Medium']%0.7)
r19 = ctrl.Rule(daw['LM'], priority['High'])
r20 = ctrl.Rule(daw['HM'], priority['Medium']%0.5)
r21 = ctrl.Rule(daw['LF'], priority['High'])
r22 = ctrl.Rule(daw['HF'], priority['Low']%0.5)
rules.append([r18, r19, r20, r21, r22])

f_r = []
for rs in rules :
    for r in rs :
        f_r.append(r)

In [4]:
fec_ctrl = ctrl.ControlSystem(f_r)
fec = ctrl.ControlSystemSimulation(fec_ctrl)

def fec_eval(inputs) :
    fec.input['waiting_time'] = inputs[0]
    fec.input['riding_time'] = inputs[1]
    fec.input['loading'] = inputs[2]
    fec.input['travelling_distance'] = inputs[3]
    fec.input['hall_call_area_weight'] = inputs[4]
    fec.input['destination_area_weight'] = inputs[5]
    
    fec.compute()
    return fec.output['priority']